In [1]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "id": "34bedada",
      "metadata": {

      },
      "source": [
        "\n",
        "# TCP/IP Encapsulation Project – Student Guide (English)\n",
        "\n",
        "**Goal:** Craft IPv4+TCP packets. On Linux/macOS, use raw sockets; on Windows, automatically fall back to **Scapy + Npcap**. \u003Cbr\u003E\n",
        "**Flow:** CSV (Application Messages) → Notebook (Encapsulation Simulation) → Wireshark (Capture) → Report (Explanation) \u003Cbr\u003E\n",
        "**Safety:** Educational use only. Prefer loopback/VM. Admin/root privileges usually required.\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "8526b6f4",
      "metadata": {

      },
      "source": [
        "## Prerequisites\n",
        "- **Linux/macOS**: `sudo jupyter lab` → capture `lo` in Wireshark → run `demo_send()`\n",
        "- **Windows**:\n",
        "  1) Install Wireshark/Npcap with *WinPcap API-compatible mode* + *Support loopback traffic*.\n",
        "  2) `pip install scapy pandas`\n",
        "  3) Run Jupyter **as Administrator**\n",
        "  4) Capture **Npcap Loopback Adapter** in Wireshark\n",
        "  5) Run `demo_send(iface='Npcap Loopback Adapter')`\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "8e0df89f",
      "metadata": {

      },
      "source": [
        "\n",
        "## Step 1 — Load Your CSV (Input)\n",
        "1. Place your CSV file (e.g., `group05_http_input.csv`) in the same folder as this notebook.\n",
        "2. The CSV must contain the following columns: `msg_id, app_protocol, src_app, dst_app, message, timestamp`.\n",
        "3. In the next cell, set `CSV_PATH` to your file name and run the cell.\n",
        "4. Verify that the preview shows your rows correctly.\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "c84b8872",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "#TODO: Load CSV file with messages into pandas DataFrame, \n",
        "\n",
        "# Replace 'path_to_your_file.csv' with the actual file path\n",
        "import pandas as pd\n",
        "filename =  \"./tcpip_project_sample.csv\" # \"path_to_your_file.csv\"\n",
        "messages_df = pd.read_csv(filename)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "339d390c",
      "metadata": {

      },
      "source": [
        "\n",
        "## Step 2 — Validate the Schema\n",
        "The notebook will automatically check the CSV header. If a required column is missing, you will see an error message.\n",
        "- If validation fails, fix your CSV and re-run Step 1.\n",
        "- If it passes, continue to Step 3.\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "c8d2219a",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "# TODO: Run the cell to validate the CSV file format\n",
        "\n",
        "def validate_csv_format(df: pd.DataFrame):\n",
        "    expected_columns = [\"app_protocol\", \"src_port\", \"dst_port\", \"message\", \"timestamp\"]\n",
        "    for col in expected_columns:\n",
        "        if col not in df.columns:\n",
        "            raise ValueError(f\"Missing expected column: {col}\")\n",
        "        \n",
        "messages_df['message'] = messages_df['message'].fillna('')  # Fill NaN messages with empty strings\n",
        "validate_csv_format(messages_df)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "94f366e9",
      "metadata": {

      },
      "source": [
        "\n",
        "## Step 3 — Map to TCP/IP Layers (Encapsulation)\n",
        "This notebook will map each application message to the TCP/IP stack:\n",
        "- Application → Transport (e.g., TCP/UDP headers)\n",
        "- Internet (IP headers)\n",
        "- Link (frame headers/footers)\n",
        "\n",
        "Just run the cell(s) in this section to see the derived structures.\n",
        "\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "d6c89854",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "# NOTE: Follow the step-by-step markdown cells above. Set paths only where indicated.\n",
        "import socket, struct, random, time, platform\n",
        "from typing import Optional\n",
        "\n",
        "IS_WINDOWS = (platform.system() == 'Windows')\n",
        "try:\n",
        "    from scapy.all import IP as SCAPY_IP, TCP as SCAPY_TCP, Raw as SCAPY_Raw, send as scapy_send, get_if_list\n",
        "    HAVE_SCAPY = True\n",
        "except Exception as e:\n",
        "    HAVE_SCAPY = False\n",
        "    SCAPY_IMPORT_ERR = e\n",
        "IS_WINDOWS, HAVE_SCAPY"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "e5596fdc",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "# Function to calculate checksum\n",
        "def checksum(data: bytes) -\u003E int:\n",
        "    if len(data) % 2:\n",
        "        data += b'\\0'\n",
        "    res = sum(struct.unpack('!%dH' % (len(data)//2), data))\n",
        "    while res \u003E\u003E 16:\n",
        "        res = (res & 0xFFFF) + (res \u003E\u003E 16)\n",
        "    return ~res & 0xFFFF\n",
        "\n",
        "# Helper function to display the data \n",
        "def hexdump(data: bytes, width: int=16):\n",
        "    for i in range(0, len(data), width):\n",
        "        chunk = data[i:i+width]\n",
        "        hex_bytes = ' '.join(f'{b:02x}' for b in chunk)\n",
        "        ascii_bytes = ''.join(chr(b) if 32 \u003C= b \u003C 127 else '.' for b in chunk)\n",
        "        print(f\"{i:04x}  {hex_bytes:\u003C{width*3}}  {ascii_bytes}\")\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "0776ecb4",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "def build_ip_header(src_ip: str, dst_ip: str, payload_len: int, proto: int=socket.IPPROTO_TCP) -\u003E bytes:\n",
        "    version_ihl = (4 \u003C\u003C 4) + 5\n",
        "    tos = 0\n",
        "    total_length = 20 + payload_len\n",
        "    identification = random.randint(0, 65535)\n",
        "    flags_fragment = 0\n",
        "    ttl = 64\n",
        "    header_checksum = 0\n",
        "    src = socket.inet_aton(src_ip)\n",
        "    dst = socket.inet_aton(dst_ip)\n",
        "    ip_header = struct.pack('!BBHHHBBH4s4s',\n",
        "                             version_ihl, tos, total_length, identification,\n",
        "                             flags_fragment, ttl, proto, header_checksum,\n",
        "                             src, dst)\n",
        "    chksum = checksum(ip_header)\n",
        "    ip_header = struct.pack('!BBHHHBBH4s4s',\n",
        "                             version_ihl, tos, total_length, identification,\n",
        "                             flags_fragment, ttl, proto, chksum,\n",
        "                             src, dst)\n",
        "    return ip_header\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "9f585b01",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "def build_tcp_header(src_ip: str, dst_ip: str, src_port: int, dst_port: int, payload: bytes=b'',\n",
        "                     seq: Optional[int]=None, ack_seq: int=0, flags: int=0x02, window: int=65535) -\u003E bytes:\n",
        "    if seq is None:\n",
        "        seq = random.randint(0, 0xFFFFFFFF)\n",
        "    doff_reserved = (5 \u003C\u003C 4)\n",
        "    checksum_tcp = 0\n",
        "    urg_ptr = 0\n",
        "    tcp_header = struct.pack('!HHLLBBHHH',\n",
        "                              src_port, dst_port, seq, ack_seq,\n",
        "                              doff_reserved, flags, window,\n",
        "                              checksum_tcp, urg_ptr)\n",
        "    placeholder = 0\n",
        "    protocol = socket.IPPROTO_TCP\n",
        "    tcp_length = len(tcp_header) + len(payload)\n",
        "    pseudo_header = struct.pack('!4s4sBBH',\n",
        "                                socket.inet_aton(src_ip), socket.inet_aton(dst_ip),\n",
        "                                placeholder, protocol, tcp_length)\n",
        "    chksum = checksum(pseudo_header + tcp_header + payload)\n",
        "    tcp_header = struct.pack('!HHLLBBH H H',\n",
        "                              src_port, dst_port, seq, ack_seq,\n",
        "                              doff_reserved, flags, window,\n",
        "                              chksum, urg_ptr)\n",
        "    return tcp_header\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "2c53cea8",
      "metadata": {

      },
      "source": [
        "### Cross‑Platform Transport\n",
        "- Linux/macOS: raw sockets (we include the IP header)\n",
        "- Windows: Scapy + Npcap fallback (raw TCP sockets are blocked by the OS)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "6655d11c",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "class RawTcpTransport:\n",
        "    def __init__(self, src_ip: str, dst_ip: str, src_port: int, dst_port: int, iface: Optional[str]=None):\n",
        "        self.src_ip = src_ip\n",
        "        self.dst_ip = dst_ip\n",
        "        self.src_port = src_port\n",
        "        self.dst_port = dst_port\n",
        "        self.iface = iface\n",
        "        self.windows_fallback = IS_WINDOWS\n",
        "        if not self.windows_fallback:\n",
        "            self.sock = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_RAW)\n",
        "        else:\n",
        "            if not HAVE_SCAPY:\n",
        "                raise RuntimeError(\n",
        "                    f\"Windows detected but Scapy is not available: {SCAPY_IMPORT_ERR}.\\n\"\n",
        "                    \"Install with: pip install scapy. Ensure Npcap is installed with loopback support.\"\n",
        "                )\n",
        "\n",
        "    def encapsulate(self, data: bytes, flags: int=0x02) -\u003E bytes:\n",
        "        tcp = build_tcp_header(self.src_ip, self.dst_ip, self.src_port, self.dst_port, data, flags=flags)\n",
        "        ip  = build_ip_header(self.src_ip, self.dst_ip, len(tcp) + len(data))\n",
        "        return ip + tcp + data\n",
        "\n",
        "    def send(self, data: bytes, flags: int=0x02):\n",
        "        if not self.windows_fallback:\n",
        "            pkt = self.encapsulate(data, flags=flags)\n",
        "            self.sock.sendto(pkt, (self.dst_ip, 0))\n",
        "        else:\n",
        "            scapy_pkt = SCAPY_IP(src=self.src_ip, dst=self.dst_ip)/SCAPY_TCP(sport=self.src_port, dport=self.dst_port, flags=flags)/SCAPY_Raw(data)\n",
        "            chosen_iface = self.iface\n",
        "            if chosen_iface is None and self.dst_ip in (\"127.0.0.1\", \"::1\"):\n",
        "                chosen_iface = \"Npcap Loopback Adapter\"\n",
        "            scapy_send(scapy_pkt, verbose=False, iface=chosen_iface)\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "299013f0",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "# find interface name for Windows\n",
        "if IS_WINDOWS and HAVE_SCAPY:\n",
        "    try:\n",
        "        print('\\n'.join(get_if_list()))\n",
        "    except Exception as e:\n",
        "        print('Could not list interfaces:', e)\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "c3f63fc6",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "# Preview packet structure\n",
        "src_ip = '127.0.0.1'\n",
        "dst_ip = '127.0.0.1'\n",
        "src_port = random.randint(1024, 65535)\n",
        "dst_port = 12345\n",
        "payload = b'Hello Packet (preview)'\n",
        "pkt_preview = build_ip_header(src_ip, dst_ip, 20 + len(payload)) + build_tcp_header(src_ip, dst_ip, src_port, dst_port, payload) + payload\n",
        "hexdump(pkt_preview)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "4f0432bc",
      "metadata": {

      },
      "source": [
        "\n",
        "## Step 4 — Capture in Wireshark\n",
        "1. Start capture in Wireshark.\n",
        "2. Run the transmit/simulation cells in this notebook.\n",
        "3. Observe packets appearing in Wireshark (timing may vary by system).\n",
        "4. Stop the capture and save the file as `.pcap`.\n",
        "\n",
        "### Suggested Wireshark Filters\n",
        "- `ip.addr == 127.0.0.1 && tcp.port == 12345`\n",
        "- `tcp.flags.syn == 1 && tcp.flags.ack == 0 && tcp.port == 12345`\n",
        "- `tcp.flags.push == 1 && tcp.flags.ack == 1 && tcp.port == 12345`\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "0e2e9a2c",
      "metadata": {

      },
      "source": [
        "\n",
        "## Step 5 — Generate/Synthesize Traffic\n",
        "The notebook will simulate the transmission of your messages. You do not need to change parameters unless instructed in comments.\n",
        "- Make sure Wireshark is open and ready to capture on your active interface.\n",
        "- Consider applying a simple filter (e.g., `tcp port 80` for HTTP) to focus the view.\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "bc6f4233",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "# Create transport instance\n",
        "src_ip = '127.0.0.1'\n",
        "dst_ip = '127.0.0.1'\n",
        "src_port = random.randint(1024, 65535)\n",
        "dst_port = 12345\n",
        "iface = \"\\\\Device\\\\NPF_Loopback\"  # Set to \"Npcap Loopback Adapter\" on Windows if needed\n",
        "transport = RawTcpTransport(src_ip, dst_ip, src_port, dst_port, iface=iface)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "cc94cac5",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "def demo_send(num_packets: int=3, delay_sec: float=1.0, flags: int=0x02):\n",
        "    for i in range(num_packets):\n",
        "        payload = f'Hello Packet {i}'.encode()\n",
        "        transport.send(payload, flags=flags)\n",
        "        time.sleep(delay_sec)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "4ca8829c",
      "metadata": {

      },
      "source": [
        "### Run (commented for safety)\n",
        "- Linux/macOS: `demo_send()`\n",
        "- Windows loopback: `demo_send(iface='Npcap Loopback Adapter')`\n",
        "- Try flags: `flags=0x18` (PSH+ACK), `0x10` (ACK), `0x01` (FIN), `0x04` (RST)\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "d00515e3",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "# demo_send(num_packets=3, delay_sec=1.0, flags=0x02)\n",
        "# demo_send(num_packets=3, flags=0x18)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "61311c01",
      "metadata": {

      },
      "source": [
        "### Send Messages from CSV file \n",
        "\n",
        "Iterate over the rows and send message by message"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "id": "45df6e75",
      "metadata": {

      },
      "outputs": [],
      "source": [
        "#Send messages from CSV file\n",
        "for index, row in messages_df.iterrows():\n",
        "    # Extract message details from the DataFrame row\n",
        "    message = row['message']\n",
        "    message = f\"test message {index}\" if not message else message\n",
        "    # Send the message using the RawTcpTransport class\n",
        "    # (You may need to adjust flags and other parameters as needed)\n",
        "    \n",
        "    #TODO: uncomment the line below to send the messages\n",
        "    transport.send(message.encode(), flags=0x18)  # Example with PSH+ACK flags\n",
        "    \n",
        "    time.sleep(0.1)  # Optional delay between messages"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "d590065f",
      "metadata": {

      },
      "source": [
        "\n",
        "## Step 6 — Analyze and Explain\n",
        "In your **report**:\n",
        "- Explain how the CSV application messages became packets/frames through encapsulation.\n",
        "- Use **Wireshark screenshots** to illustrate headers, ports, and payloads.\n",
        "- Link observations back to your CSV rows (e.g., message IDs).\n"
      ]
    },
    {
      "cell_type": "markdown",
      "id": "09e13f35",
      "metadata": {

      },
      "source": [
        "\n",
        "## Deliverables Checklist\n",
        "- [ ] CSV input file.\n",
        "- [ ] Executed notebook (with outputs).\n",
        "- [ ] Wireshark .pcap capture.\n"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "dl",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.12.3"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}

NameError: name 'null' is not defined